### Import the necessary packages and set the style sheet for this notebook
Setting the stylesheet here to display the dataframe in a readable manner

In [361]:
import pandas as pd
import glob
import numpy as np
import re
%matplotlib inline
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

### Examine the data dictionary

In [362]:
data_dictionary = pd.read_excel("data/CollegeScorecardDataDictionary.xlsx",sheet_name='data_dictionary')
cohort_map = pd.read_excel("data/CollegeScorecardDataDictionary.xlsx",sheet_name='cohort_map')

data_Details = data_dictionary.dropna(subset=['VARIABLE NAME']).iloc[:,[0,1,2,3,4]]
data_Details.columns = ['Name','Category','DevName','DataType','VariableName']

cohort_check = pd.merge(data_Details, cohort_map, left_on='VariableName',right_on = 'Variable Name').drop(['Variable Name'],axis=1)
cols = [c for c in cohort_check.columns if c[0:9] != 'MERGED_19']
# eliminate earlier cohorts
cohort_2000 = cohort_check[cols]

The analysis below is done to check which data set is relevant for our study. Based on the numbers, 2014 has most values for almost all categories, 2016 and 2017 do not have a lot of completion data. 2015, 2016 and 2017 does not have earnings data. Also rows before 2000 have very low non-null values.
2014 have lesser school data but school data can also be taken from other data sets if needed. Hence 2014 is the dataset that will be considered for our analysis

In [363]:
count_by_cat = cohort_check.groupby('Category').agg('count')
#total of all columns to check total non null values
count_by_cat.sum(axis=0) #rows before 2000 have very low null values

#remove 19's from the count dataframe
cols = [c for c in count_by_cat.columns if c[0:9] != 'MERGED_19']
count_by_cat= count_by_cat[cols]

#maxvalues = count_by_cat.loc[:,~count_by_cat.columns.isin(['Name','DevName','DataType','VariableName'])].max(axis=1)
#sum across columns 
count_by_cat["max"] = count_by_cat.loc[:,'MERGED_2000-01 datafile':'MERGED_2017-18 datafile'].max(axis=1)

count_by_cat["diff_2014"] = count_by_cat['MERGED_2014-15 datafile'] - count_by_cat["max"]
#examine for everything other than repayment and aid categories
count_by_cat.iloc[[0,1,3,4,5,7,8,9]]

,Name,DevName,DataType,VariableName,MERGED_2000-01 datafile,MERGED_2001-02 datafile,MERGED_2002-03 datafile,MERGED_2003-04 datafile,MERGED_2004-05 datafile,MERGED_2005-06 datafile,...,MERGED_2010-11 datafile,MERGED_2011-12 datafile,MERGED_2012-13 datafile,MERGED_2013-14 datafile,MERGED_2014-15 datafile,MERGED_2015-16 datafile,MERGED_2016-17 datafile,MERGED_2017-18 datafile,max,diff_2014
Category,,,,,,,,,,,,,,,,,,,,,
academics,247,247,247,247,241,241,241,241,241,241,...,247,247,247,247,247,247,247,247,247,0
admissions,25,25,25,25,0,19,19,19,19,19,...,25,25,25,25,25,25,19,19,25,0
completion,1214,1214,1214,1214,622,624,820,820,1016,1016,...,1050,1034,1034,1034,1034,1082,112,216,1082,-48
cost,77,77,77,77,9,9,9,9,9,9,...,71,47,47,47,47,47,47,47,71,-24
earnings,76,76,76,76,0,0,0,26,5,36,...,10,62,52,52,52,0,0,0,62,-10
root,5,5,5,5,3,3,3,3,3,3,...,3,3,3,3,3,3,3,5,5,-2
school,44,44,44,44,16,17,18,18,18,18,...,19,20,20,20,20,20,19,42,42,-22
student,113,113,113,113,52,68,55,68,59,72,...,63,64,63,64,63,64,63,36,72,-9


In [364]:
#Examine Null values in 2014_2015 in detail

cohort_2000[(cohort_2000.Category == 'completion') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] #has nulls but is not required
cohort_2000[(cohort_2000.Category == 'cost') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] 
cohort_2000[(cohort_2000.Category == 'root') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] # latitude and longitude is null
cohort_2000[(cohort_2000.Category == 'school') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] # there are some null columns but this can be pulled from other cohorts
cohort_2000[(cohort_2000.Category == 'student') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] # has null values but that will not affect our analysis
cohort_2000[(cohort_2000.Category == 'admissions') & (cohort_2000['MERGED_2014-15 datafile'].isnull())].iloc[:,[0,2,4,19]] #all non null values


,Name,DevName,VariableName,MERGED_2014-15 datafile


### Read the 2014_15 dataset

In [365]:
college_2014 = pd.read_csv('data\MERGED2014_15_PP.csv', na_values='PrivacySuppressed',low_memory=False)
college_2014.shape

(7703, 1977)

### Clean Up to remove unnecessary columns
1. Remove colleges with highest degree as associate or certificate as our analysis is focused on Bachelors and Graduate program
2. Remove columns will all null values

In [366]:
print("Before removing associate and certificate records:")
print(college_2014.HIGHDEG.value_counts())
print(college_2014.shape)
print("After removing associate and certificate records:")
college_2014 = college_2014[college_2014.HIGHDEG.isin([3,4,0])]
college_2014.shape

Before removing associate and certificate records:
1    2606
4    2034
2    1671
3     937
0     455
Name: HIGHDEG, dtype: int64
(7703, 1977)
After removing associate and certificate records:


(3426, 1977)

### Change the column data type
1. Update column data type to categorical wherever appropriate
2. Update column data type to numeric wherever appropriate

In [367]:
cat_index = ['MAIN','PREDDEG','HIGHDEG','CONTROL','ST_FIPS','REGION','LOCALE','LOCALE2','CCBASIC','CCUGPROF','CCSIZSET',
             'HBCU','PBI','ANNHI','TRIBAL','AANAPII','HSI','NANTI','MENONLY','WOMENONLY','RELAFFIL','DISTANCEONLY']

pattern = re.compile('CIP\d+.')

cip_columns = [col for col in college_2014.columns if bool(pattern.match(col)) == True]
cat_index= cat_index + cip_columns
print("Before setting Categorical column type")
print(college_2014.info())

cat_col_list = [item  for item in cat_index if item in college_2014.columns]

college_2014[cat_col_list] = college_2014[cat_col_list].astype('category')

Before setting Categorical column type
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3426 entries, 0 to 7702
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(1953), int64(11), object(13)
memory usage: 51.7+ MB
None


In [368]:
%%time
numeric_dtypes = (data_Details[(data_Details.DataType.isin(['integer','float'])) & ~(data_Details.VariableName.isin(cat_index)) & (data_Details.VariableName.isin(college_2014.columns))]['VariableName'])

dict_types = {i:type(3.5) for i in numeric_dtypes}
college_2014 = college_2014.astype(dict_types)
#college_2014[numeric_dtypes] = college_2014[numeric_dtypes].apply(pd.to_numeric, errors = 'coerce')
college_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3426 entries, 0 to 7702
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: category(212), float64(1750), int64(2), object(13)
memory usage: 46.9+ MB
Wall time: 318 ms


In [369]:
print("The count of columns with all null values: ",college_2014.isna().all().value_counts().loc[True]) #1977 columns
#drop all the columns with all null values
college_2014 = college_2014.dropna(how='all',axis='columns')
print("Is there columns with all null values after removing them?")
True in college_2014.isna().all().value_counts()

The count of columns with all null values:  330
Is there columns with all null values after removing them?


False

### Set the UNITID as index

In [370]:
print("check if there are duplicate UNITIDs")
print((college_2014.duplicated(subset=['UNITID'], keep='first')).value_counts())
college_2014.set_index(['UNITID'],inplace=True)
college_2014.info()

check if there are duplicate UNITIDs
False    3426
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3426 entries, 100654 to 48414501
Columns: 1646 entries, OPEID to MTHCMP6
dtypes: category(197), float64(1432), int64(4), object(13)
memory usage: 38.6+ MB


### Create Column Counts Dataframe

In [371]:
def return_counts(college):
    counts_c = college.count().sort_values().to_frame().reset_index()
    counts_c.columns = ['VariableName','NumValues']
    
    counts_c = pd.merge(data_Details,counts_c, left_on='VariableName',right_on='VariableName').iloc[:,[0,1,2,4,5]]
    return counts_c

In [372]:
print(college_2014.shape)

counts_check = return_counts(college_2014)
   
list(counts_check[counts_check.NumValues <500]['VariableName'])
college_2014 = college_2014.drop(list(counts_check[counts_check.NumValues <500]['VariableName']),axis=1)
print(college_2014.shape)

#recalculate counts after dropping
counts_check = return_counts(college_2014)

(3426, 1646)
(3426, 1355)


### 1. REPAYMENT and AID

Repayment and aid categories are not required for our study. Hence these two categories are dropped

In [373]:
repaymentaid_col = list(counts_check[counts_check.Category.isin(['repayment','aid'])]['VariableName'])

college_2014 = college_2014.drop(repaymentaid_col,axis=1)
college_2014.shape


(3426, 1187)

### 2. EARNINGS
<b>Drawbacks with earnings data:</b>
1. Data is not yet available to produce program-level earnings data. Research suggests that the variation across programs within an institution may be even greater than aggregate earnings across institutions; for instance, STEM and health majors frequently earn more than students who study in other fields. 
2. Also the data include only Title IV-receiving students, so figures may not be representative of institutions with a low proportion of Title IV-eligible students.

<b>Data Cleaning:</b>
6yrs data and 10yrs data have most non null values. These measures will be retained from 6yr and 10yr for the analysis:
1. Mean earnings of male/female students working and also the count
2. Mean/Median/Std Deviation of students working and also the count
3. Mean earnings of students working in all income terciles and also the count
4. Share of students earning over 28k/year
5. number of students not working


In [374]:
print(college_2014.shape)
#examine the earnings col
counts_check[counts_check.Category == 'earnings'].sort_values(by='NumValues')

earnings_col = list(counts_check[(counts_check.Category == 'earnings') & (counts_check.VariableName.str.contains('P8|INDEP'))]['VariableName'])

college_2014 = college_2014.drop(earnings_col,axis=1)

print(college_2014.shape)

(3426, 1187)
(3426, 1171)


### 3. STUDENT
The students' self report race and gender data will be retained for the analysis. The number of degree seeking undergraduates, average age of entry and number of graduate students are also retained.


In [375]:
stud_col = list(counts_check[(counts_check.Category == 'student') & ~counts_check.VariableName.str.startswith('INC_PCT') & ~counts_check.DevName.str.contains('age_entry|size|grad_students|race_ethni|demographics.men|demographics.women')]['VariableName'])

college_2014 = college_2014.drop(stud_col,axis=1)
college_2014.shape


(3426, 1129)

### 4. ADMISSIONS
Only the SAT AVG OVERALL and SAT Averages by OPEID will be retained. The ACT scores have large null values.


In [376]:
pd.options.display.max_rows=1000

#remove all the admissions columns except SAT average and SAT avegerage overall and admission rates. Because ACT scores has more null values
admissions_col = list(counts_check[(counts_check.Category == 'admissions') & (~counts_check.VariableName.isin(['SAT_AVG','SAT_AVG_ALL','ADM_RATE','ADM_RATE_ALL']))]['VariableName'])
college_2014 = college_2014.drop(admissions_col,axis=1)

### 5. COMPLETION
Only Title IV completion numbers will be retained since the earnings are reported for only Title IV students

In [377]:
# Remove title iv columns that are not required
titleiv_col = list(counts_check[(counts_check.Category == 'completion') & counts_check.DevName.str.contains('title_iv') & counts_check.DevName.str.contains('transf|unknown|died|pell|first_gen|loan|depend|enrolled|withdrawn')]['VariableName'])

college_2014 = college_2014.drop(titleiv_col,axis=1)

#recrete the counts
counts_check = return_counts(college_2014)

#Remove completion columns that are not required
compl_col = list(counts_check[counts_check.Category.isin(['completion']) & counts_check.DevName.str.contains('dependent|pell|loan|first_gen|separation|transfer')]['VariableName'])

college_2014 = college_2014.drop(compl_col,axis=1)
print(college_2014.shape)

#recrete the counts
counts_check = return_counts(college_2014)


(3426, 425)


In [378]:
#recheck titleiv columns
pd.set_option('mode.chained_assignment', None)

titleiv = counts_check[counts_check.Category == 'completion']
titleiv['CatName'] = titleiv['DevName'].str[:-5].str[9:]
titleiv['time'] = titleiv['DevName'].str[-4:]
titleiv = titleiv.iloc[:,[0,2,3,4,5]]

pd.reset_option('mode.chained_assignment')
titleiv.head()

,Name,DevName,VariableName,NumValues,CatName
294,"Completion rate for first-time, full-time students at four-year institutions (150% of expected time to completion)",completion_rate_4yr_150nt,C150_4,2434,n_rate_4yr_
295,Adjusted cohort count for completion rate at four-year institutions (denominator of 150% completion rate),completion_cohort_4yr_150nt,D150_4,2434,n_cohort_4yr_
296,"Completion rate for first-time, full-time students at four-year institutions (150% of expected time to completion) for white students",completion_rate_4yr_150_white,C150_4_WHITE,2318,n_rate_4yr_150_
297,"Completion rate for first-time, full-time students at four-year institutions (150% of expected time to completion) for black students",completion_rate_4yr_150_black,C150_4_BLACK,2157,n_rate_4yr_150_
298,"Completion rate for first-time, full-time students at four-year institutions (150% of expected time to completion) for Hispanic students",completion_rate_4yr_150_hispanic,C150_4_HISP,2175,n_rate_4yr_150_his


### 6. ACADEMICS
The CERTIFICATE and ASSOCIATE information can be removed as the analysis is focussed on 

In [379]:
#Exploring using RE for removing columns
pattern = re.compile(r'CIP\d+[CERTASSOC].')
#academics_col = list(counts_check[(counts_check.Category == 'academics') & (counts_check.VariableName.str.contains('CERT|ASSOC'))]['VariableName'])
cip_columns = [col for col in college_2014.columns if bool(pattern.match(col)) == True]

college_2014 = college_2014.drop(cip_columns, axis=1)
college_2014.shape

(3426, 273)

### 6. COST
Merge the public and private columns as they are mutually exclusive. Remove private columns and then rename public columns to a generic column name.

In [380]:
pattern = re.compile('NPT4.*PUB')

npt_pubcolumns = [col for col in college_2014.columns if bool(pattern.match(col)) == True]
pattern = re.compile('NPT4.*PRIV')
npt_privcolumns = [col for col in college_2014.columns if bool(pattern.match(col)) == True]

college_2014["sum_pub"] = college_2014[npt_pubcolumns].sum(axis=1)
college_2014["sum_priv"] = college_2014[npt_privcolumns].sum(axis=1)

#private columns are not present for public controls
college_2014[(college_2014.sum_priv !=0) & (college_2014.CONTROL == 1)]
#public columns are not present for private controls
college_2014[(college_2014.sum_pub !=0) & ((college_2014.CONTROL == 3) | (college_2014.CONTROL == 2))]

#so copy public count to private columns
for pubcolumn,privcolumn  in zip(npt_pubcolumns, npt_privcolumns):
    college_2014[pubcolumn].fillna(college_2014[privcolumn],inplace=True)# = college_2014[privcolumn] + college_2014[pubcolumn]
    college_2014.rename(columns={pubcolumn: pubcolumn[0:-4]},inplace=True)

college_2014 = college_2014.drop(["sum_priv","sum_pub"],axis=1)    
college_2014 = college_2014.drop(npt_privcolumns, axis=1)
college_2014.shape

(3426, 264)

In [381]:
data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"DevName"] = data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"DevName"].str.replace("public","pubpriv")
data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"Name"] = data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"DevName"].str.replace("public","pubpriv")

data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"VariableName"] = data_Details.loc[data_Details.VariableName.isin(npt_pubcolumns),"VariableName"].str[0:-4]
data_Details.loc[data_Details.DevName.str.contains('pubpriv'),:] 

,Name,Category,DevName,DataType,VariableName
558,avg_net_price.pubpriv,cost,avg_net_price.pubpriv,integer,NPT4
562,net_price.pubpriv.by_income_level.0-30000,cost,net_price.pubpriv.by_income_level.0-30000,integer,NPT41
563,net_price.pubpriv.by_income_level.30001-48000,cost,net_price.pubpriv.by_income_level.30001-48000,integer,NPT42
564,net_price.pubpriv.by_income_level.48001-75000,cost,net_price.pubpriv.by_income_level.48001-75000,integer,NPT43
565,net_price.pubpriv.by_income_level.75001-110000,cost,net_price.pubpriv.by_income_level.75001-110000,integer,NPT44
566,net_price.pubpriv.by_income_level.110001-plus,cost,net_price.pubpriv.by_income_level.110001-plus,integer,NPT45
582,net_price.pubpriv.by_income_level.0-48000,cost,net_price.pubpriv.by_income_level.0-48000,integer,NPT4_048
586,net_price.pubpriv.by_income_level.30001-75000,cost,net_price.pubpriv.by_income_level.30001-75000,integer,NPT4_3075
588,net_price.pubpriv.by_income_level.75000-plus,cost,net_price.pubpriv.by_income_level.75000-plus,integer,NPT4_75UP


### 7. Analyse other categories
<b>Root: </b> This will be needed<br>
<b>School: </b> This will be needed<br>

In [382]:
pd.set_option('max_colwidth', 230)

counts_check[counts_check.Category == 'root'].sort_values(by='NumValues') #root column has two non-null values
counts_check[counts_check.Category == 'school'].sort_values(by='NumValues') #school has several non null columns
counts_check.shape

(425, 5)

In [383]:
college_2014["ZIP"] = college_2014["ZIP"].str[0:5]
college_2014["ZIP"] = college_2014["ZIP"].apply(pd.to_numeric, errors = 'coerce')
college_2014 = college_2014.drop(['SCH_DEG'],axis=1)

In [402]:
pd.set_option('mode.chained_assignment', None)
lookup_value = data_dictionary.iloc[:,[4,5,6]]
lookup_value.columns = ['VariableName','Value','Label']
lookup_value.loc[:,"VariableName"] = lookup_value.loc[:,"VariableName"].fillna(method='ffill')
category_col = ['MAIN','PREDDEG','HIGHDEG','CONTROL','ST_FIPS','REGION','DISTANCEONLY','ICLEVEL','OPENADMP']


pd.reset_option('mode.chained_assignment')

lookup_value[lookup_value['VariableName'].isin(category_col)]
#lookup_value#.head()

,VariableName,Value,Label
12,MAIN,0.0,Not main campus
13,MAIN,1.0,Main campus
15,PREDDEG,0.0,Not classified
16,PREDDEG,1.0,Predominantly certificate-degree granting
17,PREDDEG,2.0,Predominantly associate's-degree granting
18,PREDDEG,3.0,Predominantly bachelor's-degree granting
19,PREDDEG,4.0,Entirely graduate-degree granting
20,HIGHDEG,0.0,Non-degree-granting
21,HIGHDEG,1.0,Certificate degree
22,HIGHDEG,2.0,Associate degree


In [385]:
writer = pd.ExcelWriter('data\Scorecard_2014_15.xlsx',engine = 'xlsxwriter')
counts_check = return_counts(college_2014)
college_2014 = college_2014.reset_index()
college_2014.to_excel(writer,sheet_name='data',index=False)
counts_check.to_excel(writer,sheet_name='Category', index=False)
writer.save()
writer.close()

In [386]:
# Order columns in order of groups, change categorical numbers to values